Notebook 1: vqc_input_anomaly_analysis.ipynb   (Think of this as a calculator before training the VQC classifier and in the next notebook as we will do a trained version in the future we can these two note books as calculators for anamoly detection and learn more as we grow, Notebook 2 would be 
Purpose: Setup, input encoding, anomaly testing, output distribution analysis
 Notebook 2: vqc_classifier_training.ipynb in the next book under the same folder structure)
Use: Analyze new inputs manually, observe quantum fingerprints

Includes:

Circuit visualization

Manual feature perturbation

Pre-training probability shifts

Diagnostic interpretation (entanglement breaks, symmetry drifts)

Circuit Input Encoding Template 
Goal: Quantum Encode Graph Features
Since we don’t have real graph data yet, we’ll simulate abstract feature vectors like:

Message size

Path length

Entropy

Timing skew

These will be encoded into quantum states using rotation gates, typically RY, RZ, or RX.

In [1]:
import cirq
import sympy
import numpy as np

# Qubits
q0, q1 = cirq.LineQubit.range(2)

# Symbols (for optimization later)
theta_0 = sympy.Symbol('theta_0')
theta_1 = sympy.Symbol('theta_1')

# Input feature encoding
def encode_input(circuit, qubits, features):
    circuit.append([
        cirq.ry(features[0])(qubits[0]),
        cirq.ry(features[1])(qubits[1])
    ])

# Variational layer
def add_vqc_layer(circuit, qubits, thetas):
    circuit.append([
        cirq.CNOT(qubits[0], qubits[1]),
        cirq.ry(thetas[0])(qubits[0]),
        cirq.rz(thetas[1])(qubits[1]),
    ])

# Circuit
circuit = cirq.Circuit()

# Simulate a "normal" flow input (we'll vary this for anomalies)
input_features = np.pi * np.array([0.25, 0.75])  # normalized features

# Build the circuit
encode_input(circuit, [q0, q1], input_features)
add_vqc_layer(circuit, [q0, q1], [theta_0, theta_1])
circuit.append(cirq.measure(q0, q1, key='result'))

print("🔧 Variational Circuit:")
print(circuit)

🔧 Variational Circuit:
0: ───Ry(0.25π)───@───Ry(theta_0)───M('result')───
                  │                 │
1: ───Ry(0.75π)───X───Rz(theta_1)───M─────────────


Encodes input features (simulated graph stats) as quantum rotations

Applies a simple entangling + rotation layer (your VQC)

Prepares for measurement, now lets simulate the circuit using Cirq simulator

In [2]:
simulator = cirq.Simulator()
resolved_circuit = cirq.resolve_parameters(circuit, {
    theta_0: 0.1,
    theta_1: -0.3
})
result = simulator.run(resolved_circuit, repetitions=100)
print("📈 VQC result:")
print(result)

📈 VQC result:
result=0000100100000100010100010000000000100100000000000100000110111010000000010000000000000000100010100100, 0110111011111011101011101111101101111011111111111011110001010101111101101011101111111111001101011111


Above is the output we can either represent it in graph histogram or a simple probability counter as below

In [8]:
from collections import Counter

# Convert results to bitstring format
bitstrings = [''.join(map(str, b)) for b in result.measurements['result']]

# Count occurrences
counts = Counter(bitstrings)

# Calculate probabilities
total_shots = sum(counts.values())
probabilities = {k: v / total_shots for k, v in counts.items()}

# Pretty print
print(" Output Probabilities:")
for outcome, prob in sorted(probabilities.items()):
    print(f"{outcome}: {prob:.3f}")

 Output Probabilities:
00: 0.090
01: 0.710
10: 0.160
11: 0.040


Inject Anomaly via Input Feature Perturbation
Let’s modify the original feature vector:

In [5]:
input_features = np.pi * np.array([0.25, 0.75])

to something slightly off — e.g., we flip the second value

In [6]:
anomalous_features = np.pi * np.array([0.25, 0.25])

In [9]:
# Create new circuit for anomalous input
anomaly_circuit = cirq.Circuit()
encode_input(anomaly_circuit, [q0, q1], anomalous_features)
add_vqc_layer(anomaly_circuit, [q0, q1], [theta_0, theta_1])
anomaly_circuit.append(cirq.measure(q0, q1, key='result'))
# Resolve params
resolved_anomaly_circuit = cirq.resolve_parameters(anomaly_circuit, {
    theta_0: 0.1,
    theta_1: -0.3
})
# Simulate
anomaly_result = simulator.run(resolved_anomaly_circuit, repetitions=100)
# Extract & print probabilities
bitstrings = [''.join(map(str, b)) for b in anomaly_result.measurements['result']]
counts = Counter(bitstrings)
total_shots = sum(counts.values())
anomaly_probabilities = {k: v / total_shots for k, v in counts.items()}

print("Anomalous Input Probabilities:")
for outcome, prob in sorted(anomaly_probabilities.items()):
    print(f"{outcome}: {prob:.3f}")

Anomalous Input Probabilities:
00: 0.710
01: 0.150
10: 0.040
11: 0.100


# Original Input Probabilities:
makefile
Copy
Edit
00: 0.090
01: 0.710   ← Dominant
10: 0.160
11: 0.040
# Anomalous Input Probabilities:
makefile
Copy
Edit
00: 0.700   ← Now dominant!
01: 0.140
10: 0.020
11: 0.140

our VQC is now clearly flipping its “confidence” from 01 to 00 after a minor input change.

That shift from 71% → 14% (for 01) and 9% → 70% (for 00) tells us the symmetry has been broken, and our circuit sees it.

#in the above anomalous input probabilities
#01 and 11 having same probabilities represent an anomaly?
The short answer: It can — if they weren't tied before.
Why?
In a symmetric, clean protocol (or graph), certain outcomes dominate predictably.

When two states (like 01 and 11) are equally likely, but weren’t in the normal scenario, it can indicate:

Loss of correlation (maybe a broken entanglement-style flow)

Degenerate behavior (two outputs that shouldn't match now do)

Leaked symmetry (the circuit is now agnostic to which path is taken)

So yes, that tie could indicate confusion or drift in the circuit's learned distribution — and that's exactly what an anomaly detector should be picking up.

What we've Achieved So Far:
✅ Validated your circuit responds to symmetry-breaking inputs
✅ Observed measurable probability shifts
✅ Built a prototype quantum anomaly detector using a toy VQC